# 경제 분석 및 예측과 데이터 지능 실습4: Vector AutoRegression Basics

OECD데이터를 활용해 VAR 분석을 실습합니다. Statsmodels을 주로 다루고, 실제 데이터에 적용해 봅니다.

References:
- [Vector Autoregression with Statsmodels](https://www.statsmodels.org/stable/vector_ar.html)
- [Sims (1986, FRB of Minneapolis)](https://www.minneapolisfed.org/research/quarterly-review/are-forecasting-models-usable-for-policy-analysis)
- [Blanchard and Quah (1989, AER)](https://uh.edu/~bsorense/BlanchardQuah1989.pdf)

## oecddatabuilder 패키지 소개

- oecddatabuilder 패키지는 OECD data explorer UI(user interface)를 통해 다운로드 하는 것이 불가능한 대용량 API를 처리하려는 목적에서 개발되었습니다.
- recipe, builder의 두 클래스로 이루어 지며 원하는 분석 데이터를 recipe에 저장해 두면 추후에 다른 국가, 다른 기간으로 동일한 데이터를 반복해서 다운로드 할 수 있습니다.
- 데이터의 양이 많은 경우에 latency 조절, 트래픽 조절을 통해 block당하지 않도록 유의해야 합니다.

In [2]:
import pandas as pd
import numpy as np
import oecddatabuilder as oecd

In [ ]:
oecd.test_api_connection() # 연결을 테스트 합니다.

INFO:oecddatabuilder.utils:API connection successful.


### Recipe

- 원하는 데이터를

In [5]:
loader = oecd.RecipeLoader()

In [13]:
recipe = loader.load("DEFAULT")

INFO:oecddatabuilder.recipe_loader:User configuration merged for group 'DEFAULT'.


In [19]:
builder = oecd.OECDAPI_Databuilder(config=recipe, start="1990-Q1", end = "2024-Q4", freq = "Q",
                                   base_url="https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAMAIN1@DF_QNA,1.1/",
                                   request_interval=30)

INFO:oecddatabuilder.databuilder:Combined countries from configuration: ['CAN', 'CHN', 'DEU', 'FRA', 'GBR', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'MEX', 'USA']


### Recipe기준으로 데이터를 가져옵니다.

- 보통 호출 간격을 30초, chunk size를 100 정도 하면 10분내에 12개국의 1500행정도 데이터를 받을 수 있습니다.
- 더 자주, 더 많은 데이터를 가져올 수도 있으나 중간에 block 되어 데이터 일부를 받지 못하는 점을 방지하기 위해 안정적인 값을 설정하는 것이 좋습니다.

In [20]:
builder.fetch_data(chunk_size=100)

INFO:oecddatabuilder.databuilder:For indicator 'Y', processing time chunks: [('1990-Q1', '2014-Q4'), ('2015-Q1', '2024-Q4')]
INFO:oecddatabuilder.databuilder:Fetching data for 'Y' using URL: https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAMAIN1@DF_QNA,1.1/Q..KOR+CAN+USA+CHN+GBR+DEU+FRA+JPN+ITA+IND+MEX+IRL.S1..B1GQ....USD_PPP.LR..
INFO:oecddatabuilder.databuilder:Data for indicator 'Y' saved to /Users/minkeychang/forecasting/practices/datasets/OECD/Y.csv
INFO:oecddatabuilder.databuilder:For indicator 'C', processing time chunks: [('1990-Q1', '2014-Q4'), ('2015-Q1', '2024-Q4')]
INFO:oecddatabuilder.databuilder:Fetching data for 'C' using URL: https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAMAIN1@DF_QNA,1.1/Q..KOR+CAN+USA+CHN+GBR+DEU+FRA+JPN+ITA+IND+MEX+IRL.S1M..P3....USD_PPP.LR..
INFO:oecddatabuilder.databuilder:Data for indicator 'C' saved to /Users/minkeychang/forecasting/practices/datasets/OECD/C.csv
INFO:oecddatabuilder.databuilder:For indicator 'G', processing ti

In [21]:
df = builder.create_dataframe()
df.to_csv("../datasets/querterly_oecd_account_data.csv", index=False)
df.head()

,country,date,Y,C,G,I,EX,IM
0,CAN,1990-01-01,1015714.7,518707.4,271355.6,223461.1,207736.8,200021.9
1,DEU,1990-01-01,3220806.5,1811014.8,569128.8,725261.2,527153.8,489620.3
2,FRA,1990-01-01,2244890.8,1170205.4,554966.9,517196.6,352329.9,353194.4
3,GBR,1990-01-01,2038060.5,1141647.3,450827.1,404835.9,345478.9,331416.2
4,IRL,1990-01-01,99243.7,47489.0,21623.8,23005.3,29526.0,33508.4


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR

In [13]:
df = pd.read_csv("../datasets/querterly_oecd_account_data.csv")
countries = df["country"].unique()
print(countries)

['CAN' 'DEU' 'FRA' 'GBR' 'IRL' 'ITA' 'JPN' 'KOR' 'MEX' 'USA' 'IND']


## 벡터 자기회귀 모형(Vector AutoRegression, VAR)

- SUR(Seemingly Unrelated Regression) 하에서 GLS와 OS는 $x_{1t} = x_{2t}$인 경우에 같습니다.
- 따라서 VAR 모형은 OLS(최소제곱법)로 추정이 가능합니다.
- 소득과 소비를 VAR(2) 모형으로 표현해 보면 다음과 같습니다:

$$
\begin{bmatrix}
c_{t} \\
y_{t}
\end{bmatrix}
=
\begin{bmatrix}
u_{1} \\
u_{2}
\end{bmatrix}
+
\begin{bmatrix}
\phi_{11}(1) & \phi_{12}(1) \\
\phi_{21}(1) & \phi_{22}(1)
\end{bmatrix}
\begin{bmatrix}
c_{t-1} \\
y_{t-1}
\end{bmatrix}
+
\begin{bmatrix}
\phi_{11}(2) & \phi_{12}(2) \\
\phi_{21}(2) & \phi_{22}(2)
\end{bmatrix}
\begin{bmatrix}
c_{t-2} \\
y_{t-2}
\end{bmatrix}
+
\begin{bmatrix}
\epsilon_{1t} \\
\epsilon_{2t}
\end{bmatrix}
$$
또는
$$
\bold{X}_{t} = \mu + \Phi(1) \bold{X}_{t-1} + \Phi(2) \bold{X}_{t-2} + \Epsilon_{t}
$$

- 여기서 잔차항은 $\Epsilon_{t} \sim N(0, \Omega)$ 를 따르는 벡터입니다.
- 위 모형이 성립하려면 $| I - \Phi(1)Z - \Phi(2)Z^{2} | = 0$ 의 모든 해(solution)의 절대값이 1보다 커야 합니다.
- 이러한 정상성(stationarity) 가정은 eigenvalue(고유값)의 가장 큰 값이 1보다 작다는 말과 동일합니다.



### Lag Length(시차 길이)

1. **Lag Length 결정의 중요성:**
   - VAR 모형의 시차(lag) 길이는 모형의 설명력과 예측 성능에 크게 영향을 미칩니다.
   - 너무 짧은 시차를 선택하면 모형에 중요한 동적 정보를 반영하지 못하고, 너무 긴 시차를 선택하면 모형 복잡성이 증가하고 불필요한 파라미터가 포함됩니다.

2. **통계적 검정 방법:**
   - **Wald Test(F-test):** 
     - F 통계량:
       $$
       F = \frac{(|\hat{\Omega}(H_{0})| - |\hat{\Omega}(H_{1})|) / J}{|\hat{\Omega}(H_{1})| / (T-k)}
       $$
   - **Likelihood Ratio Test(LR Test):** 
     - LR 통계량:
       $$
       LR = (T-k) \left( \log |\hat{\Omega}(H_{0})| - \log |\hat{\Omega}(H_{1})| \right) \sim \chi^{2}(J)
       $$
       - 여기서 $\hat{\Omega}(H_{0}) = \sum \hat{\epsilon}_{t}(H_{0})\hat{\epsilon}_{t}(H_{0})'$, $\hat{\Omega}(H_{1}) = \sum \hat{\epsilon}_{t}(H_{1})\hat{\epsilon}_{t}(H_{1})'$ 는 잔차 공분산 행렬을 의미합니다.
       - T는 표본 크기, k는 추정된 파라미터의 수, 그리고 J는 제약 조건의 수를 의미합니다.
       - 또한 $\hat{\epsilon}_{t} = \bold{X}_{t} - \hat{\mu} - \Phi(1) \bold{X}_{t-1} - \Phi(2) \bold{X}_{t-2} $ 이고 가설에 따라 $\Phi(1)$ 또는 $\Phi(2)$ 가 0이 됩니다.

3. **순차적 검정 방법(Sequential Test):**
   - 시차 길이 결정을 위해 순차적 검정 방법을 사용하여, 한 단계씩 추가하면서 모형의 적합도를 평가합니다.
   - 각 단계에서 추가된 시차가 통계적으로 유의한지 여부를 평가하고, 유의하지 않은 경우 시차 길이를 결정하게 됩니다.
   - 이러한 검정 방법은 Wald test나 LR test를 기반으로 하며, 큰 표본에서는 두 검정의 결과가 동일하게 나타나는 경향이 있습니다.
   - 예를 들어, LR 검정 통계량과 J * F (F 통계량을 chi-square 분포로 변환한 통계량)은 큰 표본 환경에서 동등한 결과를 보입니다.


### Granger Causality Test

   - 위 모형에서 y가 c를 GC(Granger Cause)하지 '않는다'는 다음과 같이 검정할 수 있습니다.
   $$
   H_{0} : \phi_{12}(1) =\phi_{12}(2) = 0
   $$

   - 마찬가지로, c가 y를 GC하지 '않는다'는 것은 다음과 같이 검정합니다.
   $$
   H_{0} : \phi_{21}(1) =\phi_{21}(2) = 0
   $$

   - Test statistic은 아래와 같습니다.
   $$
   F = \frac{(RSS_{0} - RSS_{1})/J}{RSS_{1}/(T-k)} \quad \sim \quad F(T-k)
   $$




##### 실습 코드는 아래와 같습니다.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR

# -------------------------------------------------------------------
# 1. 데이터 불러오기 및 전처리
# -------------------------------------------------------------------
selected_country = 'KOR'
df_country = df[df["country"] == selected_country].copy()

# 날짜 변환 및 인덱스 설정
df_country['date'] = pd.to_datetime(df_country['date'])
df_country.set_index('date', inplace=True)

# 주어진 데이터가 분기(quarterly) 데이터임을 명시적으로 설정
df_country = df_country.asfreq('QS')

# VAR 분석에 사용할 변수 선택: 소득(Y)와 소비(C)
data = df_country[['Y', 'C']]

# -------------------------------------------------------------------
# 2. VAR 모형 적합 및 시차 선택
# -------------------------------------------------------------------
# VAR 모형 생성
model = VAR(data)

# 최대 시차(maxlags)를 8로 지정하여 각 정보 기준에 따른 최적 시차 선택 결과 확인
lag_order_res = model.select_order(maxlags=8)
print("시차 선택 결과 (Lag Order Selection):")
print(lag_order_res.summary())

# 예시: 최적 시차 길이를 2로 선택 (실제 최적 시차는 선택 결과에 따라 달라질 수 있음)
optimal_lag = 2

# VAR(2) 모형 적합
results = model.fit(optimal_lag)
print(results.summary())

# 모형의 안정성(정상성) 확인: 모든 고유값의 절대값이 1보다 작아야 정상성을 만족
print("모형의 고유값 (Eigenvalues):", results.roots)


시차 선택 결과 (Lag Order Selection):
 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0       46.83       46.88   2.183e+20       46.85
1       38.15       38.28   3.694e+16       38.20
2       38.02      38.24*   3.257e+16      38.11*
3       38.04       38.34   3.305e+16       38.16
4       38.03       38.42   3.271e+16       38.19
5       38.03       38.51   3.292e+16       38.23
6       38.02       38.59   3.265e+16       38.25
7      38.02*       38.67  3.255e+16*       38.29
8       38.03       38.77   3.279e+16       38.33
-------------------------------------------------
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 12, Apr, 2025
Time:                     22:21:39
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    38.1621
Nobs:

##### 이제 로그 1차 차분을 하고 다시 분석을 수행해 보겠습니다.

In [20]:
# -------------------------------------------------------------------
# 1. 로그 변환 및 1차 차분 적용
# -------------------------------------------------------------------
# 로그 변환: 데이터의 multiplicative 효과를 additive 효과로 변환
data_log = np.log(data)

# 로그 1차 차분: 성장률(상대 변화)을 계산, small changes에 대해 (X_t - X_(t-1)) / X_(t-1) 로 근사됨
data_log_diff = data_log.diff().dropna()

# -------------------------------------------------------------------
# 2. VAR 모형 적합 및 시차 선택
# -------------------------------------------------------------------
# VAR 모형 생성 (로그 1차 차분된 데이터를 사용)
model = VAR(data_log_diff)

# 최대 시차(maxlags)를 8로 지정하여 각 정보 기준에 따른 최적 시차 선택 결과 확인
lag_order_res = model.select_order(maxlags=8)
print("시차 선택 결과 (Lag Order Selection):")
print(lag_order_res.summary())

시차 선택 결과 (Lag Order Selection):
 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -17.63      -17.58   2.212e-08      -17.61
1     -17.82*     -17.69*  1.817e-08*     -17.77*
2      -17.77      -17.55   1.920e-08      -17.68
3      -17.73      -17.42   1.991e-08      -17.61
4      -17.69      -17.29   2.086e-08      -17.53
5      -17.71      -17.23   2.041e-08      -17.51
6      -17.65      -17.08   2.155e-08      -17.42
7      -17.62      -16.97   2.223e-08      -17.36
8      -17.57      -16.83   2.339e-08      -17.27
-------------------------------------------------


In [21]:
# 예시: 최적 시차 길이를 2로 선택
optimal_lag = 2

# VAR(2) 모형 적합
results = model.fit(optimal_lag)
print(results.summary())

# 모형의 안정성(정상성) 확인: 모든 고유값(eigenvalues)의 절대값이 1보다 작아야 정상성을 만족
print("모형의 고유값 (Eigenvalues):", results.roots)

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 12, Apr, 2025
Time:                     22:23:37
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -17.5805
Nobs:                     137.000    HQIC:                  -17.7071
Log likelihood:           840.077    FPE:                1.87213e-08
AIC:                     -17.7937    Det(Omega_mle):     1.74261e-08
--------------------------------------------------------------------
Results for equation Y
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.006698         0.001562            4.289           0.000
L1.Y          0.182432         0.136190            1.340           0.180
L1.C          0.192356         0.088000            2.186           0.029
L2.Y          0.093548 

In [26]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR

# -------------------------------------------------------------------
# 1. 데이터 불러오기 및 전처리
# -------------------------------------------------------------------
selected_country = 'FRA'
df_country = df[df["country"] == selected_country].copy()

# 날짜 변환 및 인덱스 설정
df_country['date'] = pd.to_datetime(df_country['date'])
df_country.set_index('date', inplace=True)

# 데이터가 분기(quarterly) 데이터임을 명시적으로 설정 (예: quarter start)
df_country = df_country.asfreq('QS')

# 예를 들어, 1990년부터 2006년까지의 구간을 선택
df_sub = df_country.loc["1990-01-01":"2020-12-31"]

# VAR 분석에 사용할 변수 선택: 소득(Y)와 소비(C)
data = df_sub[['Y', 'C']]

# -------------------------------------------------------------------
# 2. 로그 변환 및 1차 차분 적용
# -------------------------------------------------------------------
data_log = np.log(data)
data_log_diff = data_log.diff().dropna()

# -------------------------------------------------------------------
# 3. VAR 모형 적합 및 시차 선택
# -------------------------------------------------------------------
model = VAR(data_log_diff)
lag_order_res = model.select_order(maxlags=8)
print("시차 선택 결과 (Lag Order Selection):")
print(lag_order_res.summary())

optimal_lag = 2  # 예시: 최적 시차 길이를 2로 선택

results = model.fit(optimal_lag)
print(results.summary())

# 모형의 고유값 확인
eigenvalues = results.roots
print("모형의 고유값 (Eigenvalues):", eigenvalues)

# -------------------------------------------------------------------
# 4. Stationarity Check
# -------------------------------------------------------------------
# 정상성을 만족하려면 모든 고유값의 절대값이 1보다 작아야 합니다.
stationary = np.all(np.abs(eigenvalues) < 1)
if stationary:
    print("VAR 모형은 정상(stationary)입니다.")
else:
    print("VAR 모형은 비정상(non-stationary)입니다.")

# -------------------------------------------------------------------
# 5. Granger 인과성(F-test) 검정
# -------------------------------------------------------------------
# (a) Y의 변화율이 C의 변화율에 대해 Granger 인과성이 있는지 검정
gc_y_to_c = results.test_causality(caused='C', causing='Y', kind='f')
print("Granger 인과성 F-test (Y -> C):")
print(gc_y_to_c.summary())

# (b) C의 변화율이 Y의 변화율에 대해 Granger 인과성이 있는지 검정
gc_c_to_y = results.test_causality(caused='Y', causing='C', kind='f')
print("Granger 인과성 F-test (C -> Y):")
print(gc_c_to_y.summary())


시차 선택 결과 (Lag Order Selection):
 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -17.55      -17.50   2.383e-08      -17.53
1      -18.47      -18.32   9.556e-09      -18.41
2     -18.57*     -18.33*  8.645e-09*     -18.47*
3      -18.51      -18.18   9.151e-09      -18.37
4      -18.46      -18.03   9.605e-09      -18.29
5      -18.42      -17.89   1.005e-08      -18.20
6      -18.37      -17.75   1.052e-08      -18.12
7      -18.35      -17.63   1.080e-08      -18.06
8      -18.29      -17.47   1.149e-08      -17.96
-------------------------------------------------
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 12, Apr, 2025
Time:                     22:34:43
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -18.3949
Nobs:

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.tsatools import detrend

# 결과 저장용 리스트: 정상성을 달성한 국가만 저장
stationary_countries = []

# 'country' 컬럼의 고유값을 모두 가져옵니다.
countries = df["country"].unique()

# 분석할 시간 구간
start_date = "1990-01-01"
end_date   = "2024-12-31"

# 함수: VAR 모형을 적합하고 고유값을 이용하여 정상성(stationarity)을 판별
def check_stationarity(transform_data, country, method, optimal_lag=2):
    try:
        model = VAR(transform_data)
        results = model.fit(optimal_lag)
        eigenvalues = results.roots
        stationary = np.all(np.abs(eigenvalues) < 1)
        print(f"  {country} - {method}: 모형의 고유값 (Eigenvalues): {eigenvalues}")
        if stationary:
            print(f"  {country} - {method}: Stationary achieved.")
            return stationary, eigenvalues
        else:
            print(f"  {country} - {method}: Not stationary.")
            return stationary, eigenvalues
    except Exception as e:
        print(f"  {country} - {method}: VAR 모형 적합 중 에러: {e}")
        return False, None

for country in countries:
    print(f"\n===========================\n현재 국가: {country}")
    
    # 해당 국가 데이터 추출 및 전처리
    df_country = df[df["country"] == country].copy()
    df_country['date'] = pd.to_datetime(df_country['date'])
    df_country.set_index('date', inplace=True)
    df_country = df_country.asfreq('QS')  # Quarter Start frequency
    
    df_sub = df_country.loc[start_date:end_date]
    
    if not {'Y', 'C'}.issubset(df_sub.columns):
        print(f"  {country}: 필요한 변수(Y, C)가 없습니다. 건너뜁니다.")
        continue
    
    if df_sub.shape[0] < 30:
        print(f"  {country}: 관측치가 부족합니다 ({df_sub.shape[0]}개). 건너뜁니다.")
        continue
    
    data = df_sub[['Y', 'C']]
    
    try:
        data_log = np.log(data)
    except Exception as e:
        print(f"  {country}: 로그 변환 에러: {e}")
        continue
    
    # 여러 변환 방법을 딕셔너리에 저장
    transforms = {}
    # Method 1: 로그 1차 차분
    transforms["log_diff_1"] = data_log.diff().dropna()
    # Method 2: 로그 2차 차분
    transforms["log_diff_2"] = data_log.diff().diff().dropna()
    # Method 3: 로그 데이터에서 추세 제거 후 1차 차분
    data_log_detrended = pd.DataFrame(
        data_log.apply(lambda x: detrend(x, order=1)),
        index=data_log.index,
        columns=data_log.columns
    )
    transforms["detrend_log_diff_1"] = data_log_detrended.diff().dropna()
    # Method 4: 계절 차분 (분기 데이터의 경우 lag=4) - 로그 차분
    transforms["log_diff_seasonal"] = data_log.diff(4).dropna()
    
    found_stationary = False
    for method_name, transform_data in transforms.items():
        if transform_data.shape[0] < 30:
            print(f"  {country} - {method_name}: 관측치가 부족합니다.")
            continue
        
        stationary, eigenvals = check_stationarity(transform_data, country, method_name, optimal_lag=2)
        if stationary:
            stationary_countries.append((country, method_name, eigenvals))
            found_stationary = True
            break  # 정상성을 만족하는 변환 방법이 발견되면, 추가 변환 시도는 건너뜁니다.
    
    if not found_stationary:
        print(f"  {country}: 모든 변환 방법에서 비정상(non-stationary) 결과.")

# 최종적으로 정상성을 달성한 국가들만 출력
print("\n=== Stationary Countries ===")
for country, method_name, eigenvals in stationary_countries:
    print(f"{country} - {method_name}: Stationary (Eigenvalues: {eigenvals})")
  
# stationary_countries 리스트에는 정상성을 달성한 국가 및 적용된 변환 방법과 고유값 정보가 저장됩니다.



현재 국가: CAN
  CAN - log_diff_1: 모형의 고유값 (Eigenvalues): [ 1.28485294+3.03576271j  1.28485294-3.03576271j -0.94611404+1.98793715j
 -0.94611404-1.98793715j]
  CAN - log_diff_1: Not stationary.
  CAN - log_diff_2: 모형의 고유값 (Eigenvalues): [-0.70136208+1.58144634j -0.70136208-1.58144634j -0.97432067+1.1446927j
 -0.97432067-1.1446927j ]
  CAN - log_diff_2: Not stationary.
  CAN - detrend_log_diff_1: 모형의 고유값 (Eigenvalues): [ 1.28485294+3.03576271j  1.28485294-3.03576271j -0.94611404+1.98793715j
 -0.94611404-1.98793715j]
  CAN - detrend_log_diff_1: Not stationary.
  CAN - log_diff_seasonal: 모형의 고유값 (Eigenvalues): [-5.7134084   2.30695115  1.83334879  1.56797716]
  CAN - log_diff_seasonal: Not stationary.
  CAN: 모든 변환 방법에서 비정상(non-stationary) 결과.

현재 국가: DEU
  DEU - log_diff_1: 모형의 고유값 (Eigenvalues): [-6.99397853-0.j          3.41450576-0.j         -0.35799357+1.64609746j
 -0.35799357-1.64609746j]
  DEU - log_diff_1: Not stationary.
  DEU - log_diff_2: 모형의 고유값 (Eigenvalues): [-1.328812 +1.6592550

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR

# -------------------------------------------------------------------
# 1. 데이터 불러오기 및 전처리
# -------------------------------------------------------------------
selected_country = 'KOR'
df_country = df[df["country"] == selected_country].copy()

# 날짜 변환 및 인덱스 설정
df_country['date'] = pd.to_datetime(df_country['date'])
df_country.set_index('date', inplace=True)

# 주어진 데이터가 분기(quarterly) 데이터임을 명시적으로 설정
df_country = df_country.asfreq('QS')

# VAR 분석에 사용할 변수 선택: 소득(Y)와 소비(C)
data = df_country[['Y', 'C']]

# -------------------------------------------------------------------
# 2. VAR 모형 적합 및 시차 선택
# -------------------------------------------------------------------
# VAR 모형 생성
model = VAR(data)

# 최대 시차(maxlags)를 8로 지정하여 각 정보 기준에 따른 최적 시차 선택 결과 확인
lag_order_res = model.select_order(maxlags=8)
print("시차 선택 결과 (Lag Order Selection):")
print(lag_order_res.summary())

# 예시: 최적 시차 길이를 2로 선택 (실제 최적 시차는 선택 결과에 따라 달라질 수 있음)
optimal_lag = 2

# VAR(2) 모형 적합
results = model.fit(optimal_lag)
print(results.summary())

# 모형의 안정성(정상성) 확인: 모든 고유값의 절대값이 1보다 작아야 정상성을 만족
print("모형의 고유값 (Eigenvalues):", results.roots)


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR

# -------------------------------------------------------------------
# 1. 데이터 불러오기 및 전처리
# -------------------------------------------------------------------
selected_country = 'FRA'
df_country = df[df["country"] == selected_country].copy()

# 날짜 변환 및 인덱스 설정
df_country['date'] = pd.to_datetime(df_country['date'])
df_country.set_index('date', inplace=True)

# 데이터가 분기(quarterly) 데이터임을 명시적으로 설정 (예: quarter start)
df_country = df_country.asfreq('QS')

# 예를 들어, 1990년부터 2006년까지의 구간을 선택
df_sub = df_country.loc["1990-01-01":"2020-12-31"]

# VAR 분석에 사용할 변수 선택: 소득(Y)와 소비(C)
data = df_sub[['Y', 'C']]

# -------------------------------------------------------------------
# 2. 로그 변환 및 1차 차분 적용
# -------------------------------------------------------------------
data_log = np.log(data)
data_log_diff = data_log.diff().dropna()

# -------------------------------------------------------------------
# 3. VAR 모형 적합 및 시차 선택
# -------------------------------------------------------------------
model = VAR(data_log_diff)
lag_order_res = model.select_order(maxlags=8)
print("시차 선택 결과 (Lag Order Selection):")
print(lag_order_res.summary())

optimal_lag = 2  # 예시: 최적 시차 길이를 2로 선택

results = model.fit(optimal_lag)
print(results.summary())

# 모형의 고유값 확인
eigenvalues = results.roots
print("모형의 고유값 (Eigenvalues):", eigenvalues)

# -------------------------------------------------------------------
# 4. Stationarity Check
# -------------------------------------------------------------------
# 정상성을 만족하려면 모든 고유값의 절대값이 1보다 작아야 합니다.
stationary = np.all(np.abs(eigenvalues) < 1)
if stationary:
    print("VAR 모형은 정상(stationary)입니다.")
else:
    print("VAR 모형은 비정상(non-stationary)입니다.")

# -------------------------------------------------------------------
# 5. Granger 인과성(F-test) 검정
# -------------------------------------------------------------------
# (a) Y의 변화율이 C의 변화율에 대해 Granger 인과성이 있는지 검정
gc_y_to_c = results.test_causality(caused='C', causing='Y', kind='f')
print("Granger 인과성 F-test (Y -> C):")
print(gc_y_to_c.summary())

# (b) C의 변화율이 Y의 변화율에 대해 Granger 인과성이 있는지 검정
gc_c_to_y = results.test_causality(caused='Y', causing='C', kind='f')
print("Granger 인과성 F-test (C -> Y):")
print(gc_c_to_y.summary())


### 위와 같이 Stationary하지 않은 결과로는 가정이 망가지기 때문에 제대로 된 분석을 시행하기 어렵습니다.

## APPENDIX

### Eigenvalue, 정상성, 특성방정식, 단위원(Unit Circle), 로그 1차 차분

1. **Eigenvalue(고유값)와 정상성:**
   - VAR 모형의 안정성(stability)은 모형에 사용된 행렬들의 고유값에 의해 결정됩니다.
   - 구체적으로, 모형의 모든 고유값의 절대값이 1 미만일 경우, 모형은 정상(stationary)으로 간주됩니다.
   - 이는 VAR 모형이 장기적으로 수렴(convergence)하며, 충격(shock)이 시간이 지남에 따라 소멸됨을 의미합니다.

2. **특성방정식(Characteristic Equation):**
   - 예를 들어, VAR(2) 모형에서는 특성방정식이 다음과 같이 주어집니다:
     $$
     \left| I - \Phi(1)Z - \Phi(2)Z^{2} \right| = 0
     $$
   - 이 방정식의 해(즉, \( Z \)에 대한 해)는 모형의 동적 특성을 결정하며, 각 해의 절대값은 모형의 안정성에 직접적인 영향을 미칩니다.

3. **Unit Circle(단위원):**
   - Unit circle는 복소평면에서 반지름이 1인 원을 의미합니다.
   - VAR 모형의 안정성 판별 시, 특성방정식의 모든 고유값이 단위원 밖(즉, 절대값이 1보다 큰 영역)에 위치해야 합니다.
   - 실제로, 모형의 모든 고유값의 절대값이 1 미만이면(즉, 단위원 내부에 있으면) 모형은 정상(stationary)하지 않으며, 외부에 위치하면 정상성을 만족합니다.

4. **Log First Differencing (로그 1차 차분):**
   - 로그 1차 차분은 시계열 데이터의 각 시점 값에 로그를 취한 후, 인접한 시점 간의 차이를 계산하는 방법입니다.
   - 수식으로 표현하면, $\Delta \log X_t = \log(X_t) - \log(X_{t-1})$ 와 같이 계산되며, 이는 작은 변화에 대해 $\frac{X_t - X_{t-1}}{X_{t-1}}$ 로 근사될 수 있습니다.
   - 이 변환은 데이터의 상대적 변화, 즉 성장률 또는 비율 변화를 나타내며, 특히 경제나 금융 데이터를 분석할 때 유용합니다.
   - 로그 1차 차분은 다음과 같은 효과를 가집니다:
     - **비정상성 제거:** 수준(levels)의 시계열 데이터는 일반적으로 추세나 단위근(unit root) 문제로 인해 비정상(non-stationary)일 수 있는데, 로그 1차 차분을 통해 이러한 비정상성을 완화할 수 있습니다.
     - **분산 안정화:** 로그 변환을 사용하면 큰 값과 작은 값 사이의 스케일 차이가 줄어들어 분산이 안정화됩니다.
     - **해석 용이성:** 로그 1차 차분은 직접적으로 성장률(또는 비율 변화)을 반영하므로, 변수들 간의 상호작용이나 동적 변화를 해석하는 데 직관적인 장점을 제공합니다.
